In [ ]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 34.5MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13408609891060191382
xla_global_id: -1
]


In [ ]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, BatchNormalization
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, SimpleRNN
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [ ]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [ ]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [ ]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
%matplotlib inline
# define scope of configs
AHB_list = [8, 9, 10]
GRU_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [5]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in GRU_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(GRU_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                if d == 1:
                                    model.add(GRU(units=c, return_sequences=False, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 2:
                                    model.add(GRU(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(GRU(units=c, return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 3:
                                    model.add(GRU(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(GRU(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(GRU(units=c, return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                model.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                yhat = yhat.reshape(yhat.shape[0],-1)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'GRU', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_layer': d, 'GRU_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 7560 ]
{
  "model": "GRU",
  "AHB_number": "AHB_9",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 13,
  "patience_list": 3,
  "MAE_normalize": 0.09522860195022087,
  "MSE_normalize": 0.009172400020328727,
  "RMSE_normalize": 0.09577264755831243,
  "MAPE_normalize": 0.10016895728592701,
  "MAE": 7434.497767857143,
  "MSE": 55905097.276471816,
  "RMSE": 7476.971129840734,
  "MAPE": 0.05099420302377632
}
Percent CPU Usage: 24.4
Percent Ram Usage: 10.3
2022-10-12 01:01:59.664024+07:00
PROGRESS: [ 2 / 7560 ]
{
  "model": "GRU",
  "AHB_number": "AHB_9",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 24,
  "patience_list": 3,
  "MAE_normalize": 0.019737422110279672,
  "MSE_normalize": 0.0007029001213914569,
  "RMSE_normalize": 0.02651226360368833,
  "MAPE_normalize": 0.020207596314294644,
  "MAE": 1540.9017857142

{
  "model": "GRU",
  "AHB_number": "AHB_9",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 17,
  "patience_list": 3,
  "MAE_normalize": 0.3379973238713242,
  "MSE_normalize": 0.11493915706617662,
  "RMSE_normalize": 0.33902677927588054,
  "MAPE_normalize": 0.3558794380012195,
  "MAE": 26387.45089285714,
  "MSE": 700545526.7647879,
  "RMSE": 26467.820589628984,
  "MAPE": 0.18108388817591434
}
Percent CPU Usage: 47.5
Percent Ram Usage: 10.9
2022-10-12 01:02:16.867113+07:00
PROGRESS: [ 6 / 7560 ]


Streaming output truncated to the last 5000 lines.
  "MAE": 2689.1785714285716,
  "MSE": 7278810.933925084,
  "RMSE": 2697.927155044236,
  "MAPE": 0.03418321480083501
}
Percent CPU Usage: 39.2
Percent Ram Usage: 17.3
2022-10-12 14:55:52.870233+07:00
PROGRESS: [ 7353 / 7560 ]
{
  "model": "GRU",
  "AHB_number": "AHB_11",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.5,
  "n_batch": 4,
  "n_steps": 5,
  "last_epoch": 35,
  "patience_list": 10,
  "MAE_normalize": 0.13529581998369536,
  "MSE_normalize": 0.018425147420813765,
  "RMSE_normalize": 0.13573926263544298,
  "MAPE_normalize": 0.14136258832415172,
  "MAE": 5533.189732142857,
  "MSE": 30817211.89515904,
  "RMSE": 5551.325237739097,
  "MAPE": 0.07033073306699425
}
Percent CPU Usage: 54.9
Percent Ram Usage: 17.3
2022-10-12 14:55:57.858705+07:00
PROGRESS: [ 7354 / 7560 ]
{
  "model": "GRU",
  "AHB_number": "AHB_11",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "GRU_hidden_units": 8,
  "dropouts": 0.5,
  "n_b

In [ ]:
name = "all_performance_GRU_5nsteps_AHB9-10-11"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_GRU_5nsteps_AHB9-10-11.json /content/drive/MyDrive/Depression
! cp /content/all_performance_GRU_5nsteps_AHB9-10-11.csv /content/drive/MyDrive/Depression

In [ ]:
df_json

,model,AHB_number,Multi_Uni,n_layer,GRU_hidden_units,dropouts,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,GRU,AHB_9,uni,1,8,0.2,2,5,13,3,0.095229,0.009172,0.095773,0.100169,7434.497768,5.590510e+07,7476.971130,0.050994
1,GRU,AHB_9,mul,1,8,0.2,2,5,24,3,0.019737,0.000703,0.026512,0.020208,1540.901786,4.284124e+06,2069.812630,0.010425
2,GRU,AHB_9,uni,1,8,0.2,4,5,30,3,0.101835,0.010429,0.102123,0.107246,7950.232143,6.356487e+07,7972.757685,0.054565
3,GRU,AHB_9,mul,1,8,0.2,4,5,15,3,0.098873,0.010205,0.101019,0.103607,7719.026786,6.219796e+07,7886.568440,0.052844
4,GRU,AHB_9,uni,1,8,0.2,8,5,17,3,0.337997,0.114939,0.339027,0.355879,26387.450893,7.005455e+08,26467.820590,0.181084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,GRU,AHB_11,mul,3,128,0.5,32,5,33,10,0.034527,0.001227,0.035029,0.035993,1412.041295,2.052294e+06,1432.582849,0.017928
7556,GRU,AHB_11,uni,3,128,0.5,64,5,30,10,0.125122,0.015725,0.125399,0.130789,5117.102679,2.630075e+07,5128.425828,0.065056
7557,GRU,AHB_11,mul,3,128,0.5,64,5,13,10,0.250920,0.063059,0.251115,0.262475,10261.857143,1.054698e+08,10269.851208,0.130511
7558,GRU,AHB_11,uni,3,128,0.5,128,5,38,10,0.073019,0.005349,0.073137,0.076563,2986.243304,8.946655e+06,2991.095916,0.038024
